In [12]:
import pandas as pd
import pymysql
from more_itertools.more import strip

# DB 연결
conn = pymysql.connect(
    host="localhost", # MariaDB 서버 주소
    port=3306,
    user="webuser",       # 사용자명
    password="webuser",  # 비밀번호
    database="DDProject",  # 사용할 데이터베이스
    charset="utf8mb4"  # 한글 지원
)



# transactional
try:
    conn.autocommit(False)
    cursor = conn.cursor()
    # 테이블 크리에이트
    create_table_sql = """CREATE TABLE IF NOT EXISTS busstop (
        id INT AUTO_INCREMENT PRIMARY KEY,
        경도 FLOAT,
        정류장번호 INT,
        버스도착안내기설치여부 VARCHAR(20),
        정류장이름 VARCHAR(100),
        정류장유형 VARCHAR(20),
        위도 FLOAT,
        정류장아이디 INT,
        지번주소 VARCHAR(100),
        지번주소거리 VARCHAR(100),
        도로명주소 VARCHAR(100),
        도로명주소거리 VARCHAR(100)
    )"""

    cursor.execute(create_table_sql)



    # 1. 데이터 로드
    df_bus = pd.read_csv('../result/서울시_버스정류장정보.csv', encoding='utf-8', header=0)
    # 2. 데이터 변환
    df_bus['bus_arvl_info_guidem_instl'] = df_bus['bus_arvl_info_guidem_instl'].str.strip()
    # 3. 날짜 변환 (문자열 → DATETIME)

    # 테이블 인서트
    placeholders = ', '.join(['%s'] * df_bus.columns.values.size)
    insert_sql = f"""
        INSERT INTO busstop (경도,정류장번호,버스도착안내기설치여부,정류장이름,정류장유형,위도,정류장아이디,지번주소,지번주소거리,도로명주소,도로명주소거리)
        VALUES ({placeholders})
    """

    data = df_bus[df_bus.columns.values].itertuples(index=False, name=None)
    cursor.executemany(insert_sql, data)
    # 변경 사항 저장
    conn.commit()

    #transactional error
except pymysql.MySQLError as e:
    print(e)
    conn.rollback()

finally:
    # 연결 종료
    cursor.close()
    conn.close()
# null_rows = df_bus[df_bus.isnull().any(axis=1)]
# null_rows

In [13]:
df_bus.dtypes

lot                           float64
crtr_no                         int64
bus_arvl_info_guidem_instl     object
crtr_nm                        object
crtr_type                      object
lat                           float64
crtr_id                         int64
_PARCEL_AD                     object
_PARCEL_DT                     object
_ROAD_AD                       object
_ROAD_DT                       object
dtype: object